# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 14 2022 11:48AM,248643,10,SOTAH0000809,Ivaoes Animal Health,Released
1,Oct 14 2022 11:45AM,248642,16,ACG-2102491561,ACG North America LLC,Released
2,Oct 14 2022 11:31AM,248640,19,60017868,"GUSA Granules USA, Inc.",Executing
3,Oct 14 2022 11:30AM,248639,22,630680,"NBTY Global, Inc.",Released
4,Oct 14 2022 11:22AM,248638,19,ACG-2102491563,ACG North America LLC,Released
5,Oct 14 2022 11:16AM,248637,19,ACG-2102491562,ACG North America LLC,Released
6,Oct 14 2022 11:07AM,248633,15,8638284,"Mizner Bioscience, LLC",Released
7,Oct 14 2022 11:03AM,248631,15,ALIA510244-01,"Alliance Pharma, Inc.",Released
8,Oct 14 2022 11:03AM,248631,15,ALIL361355-01,"Alliance Pharma, Inc.",Executing
9,Oct 14 2022 11:03AM,248631,15,30011425,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,248638,Released,1
33,248639,Released,1
34,248640,Executing,1
35,248642,Released,1
36,248643,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248638,NaN,NaN,1.0
248639,NaN,NaN,1.0
248640,NaN,1.0,NaN
248642,NaN,NaN,1.0
248643,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248096,0.0,0.0,1.0
248101,0.0,0.0,1.0
248103,0.0,0.0,1.0
248109,0.0,0.0,1.0
248554,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248096,0,0,1
248101,0,0,1
248103,0,0,1
248109,0,0,1
248554,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248096,0,0,1
1,248101,0,0,1
2,248103,0,0,1
3,248109,0,0,1
4,248554,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248096,,,1
1,248101,,,1
2,248103,,,1
3,248109,,,1
4,248554,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 14 2022 11:48AM,248643,10,Ivaoes Animal Health
1,Oct 14 2022 11:45AM,248642,16,ACG North America LLC
2,Oct 14 2022 11:31AM,248640,19,"GUSA Granules USA, Inc."
3,Oct 14 2022 11:30AM,248639,22,"NBTY Global, Inc."
4,Oct 14 2022 11:22AM,248638,19,ACG North America LLC
5,Oct 14 2022 11:16AM,248637,19,ACG North America LLC
6,Oct 14 2022 11:07AM,248633,15,"Mizner Bioscience, LLC"
7,Oct 14 2022 11:03AM,248631,15,"Alliance Pharma, Inc."
19,Oct 14 2022 10:58AM,248630,10,"Citieffe, Inc."
20,Oct 14 2022 10:57AM,248629,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 14 2022 11:48AM,248643,10,Ivaoes Animal Health,,,1
1,Oct 14 2022 11:45AM,248642,16,ACG North America LLC,,,1
2,Oct 14 2022 11:31AM,248640,19,"GUSA Granules USA, Inc.",,1,
3,Oct 14 2022 11:30AM,248639,22,"NBTY Global, Inc.",,,1
4,Oct 14 2022 11:22AM,248638,19,ACG North America LLC,,,1
5,Oct 14 2022 11:16AM,248637,19,ACG North America LLC,,,1
6,Oct 14 2022 11:07AM,248633,15,"Mizner Bioscience, LLC",,,1
7,Oct 14 2022 11:03AM,248631,15,"Alliance Pharma, Inc.",,7,5
8,Oct 14 2022 10:58AM,248630,10,"Citieffe, Inc.",,,1
9,Oct 14 2022 10:57AM,248629,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 14 2022 11:48AM,248643,10,Ivaoes Animal Health,1,,
1,Oct 14 2022 11:45AM,248642,16,ACG North America LLC,1,,
2,Oct 14 2022 11:31AM,248640,19,"GUSA Granules USA, Inc.",,1,
3,Oct 14 2022 11:30AM,248639,22,"NBTY Global, Inc.",1,,
4,Oct 14 2022 11:22AM,248638,19,ACG North America LLC,1,,
5,Oct 14 2022 11:16AM,248637,19,ACG North America LLC,1,,
6,Oct 14 2022 11:07AM,248633,15,"Mizner Bioscience, LLC",1,,
7,Oct 14 2022 11:03AM,248631,15,"Alliance Pharma, Inc.",5,7,
8,Oct 14 2022 10:58AM,248630,10,"Citieffe, Inc.",1,,
9,Oct 14 2022 10:57AM,248629,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 14 2022 11:48AM,248643,10,Ivaoes Animal Health,1,,
1,Oct 14 2022 11:45AM,248642,16,ACG North America LLC,1,,
2,Oct 14 2022 11:31AM,248640,19,"GUSA Granules USA, Inc.",,1,
3,Oct 14 2022 11:30AM,248639,22,"NBTY Global, Inc.",1,,
4,Oct 14 2022 11:22AM,248638,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 14 2022 11:48AM,248643,10,Ivaoes Animal Health,1.0,NaN,NaN
1,Oct 14 2022 11:45AM,248642,16,ACG North America LLC,1.0,NaN,NaN
2,Oct 14 2022 11:31AM,248640,19,"GUSA Granules USA, Inc.",NaN,1.0,NaN
3,Oct 14 2022 11:30AM,248639,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Oct 14 2022 11:22AM,248638,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 14 2022 11:48AM,248643,10,Ivaoes Animal Health,1.0,0.0,0.0
1,Oct 14 2022 11:45AM,248642,16,ACG North America LLC,1.0,0.0,0.0
2,Oct 14 2022 11:31AM,248640,19,"GUSA Granules USA, Inc.",0.0,1.0,0.0
3,Oct 14 2022 11:30AM,248639,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Oct 14 2022 11:22AM,248638,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3977847,97.0,12.0,9.0
15,497264,6.0,7.0,0.0
16,497269,3.0,1.0,0.0
19,745915,2.0,1.0,0.0
21,1489561,5.0,1.0,0.0
22,248639,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3977847,97.0,12.0,9.0
1,15,497264,6.0,7.0,0.0
2,16,497269,3.0,1.0,0.0
3,19,745915,2.0,1.0,0.0
4,21,1489561,5.0,1.0,0.0
5,22,248639,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,97.0,12.0,9.0
1,15,6.0,7.0,0.0
2,16,3.0,1.0,0.0
3,19,2.0,1.0,0.0
4,21,5.0,1.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,97.0
1,15,Released,6.0
2,16,Released,3.0
3,19,Released,2.0
4,21,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Completed,9.0,0.0,0.0,0.0,0.0,0.0
Executing,12.0,7.0,1.0,1.0,1.0,0.0
Released,97.0,6.0,3.0,2.0,5.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0
1,Executing,12.0,7.0,1.0,1.0,1.0,0.0
2,Released,97.0,6.0,3.0,2.0,5.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Completed,9.0,0.0,0.0,0.0,0.0,0.0
1,Executing,12.0,7.0,1.0,1.0,1.0,0.0
2,Released,97.0,6.0,3.0,2.0,5.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()